In [1]:
import time
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display,HTML
from random import randrange
from scipy.special import comb
import numpy as np
import numba as nb

In [2]:
url='https://www.janestreet.com/puzzles/robot-swimming-trials-index/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
HTML(str(soup.find('div', {'class' :'inner-wrapper'})))

Robot,Race 1,Race 2,Race 3
Automatonya,0.6,0.1,0.3
Botty,0.6,0.3,0.1
Chroma,0,1,0
Data,0.3,0.5,0.2
Electro,0.2,0.8,0
Fernandroid,0.4,0.5,0.1
Gregulator,0.5,0.5,0
Hannanobot,0,0.9,0.1
IO,0.2,0.7,0.1


In [3]:
# A some point there is a larger than 1/3 probability of there being at least one race where every other robot choses zero. 
# So if the last robot spreads evenly over the other races he will win any race where there are no other robots who choose 1.
# The probability of throwing N balls into k bins and there being a ball in every bin is as per the link below.
# https://math.stackexchange.com/questions/174674/if-n-balls-are-thrown-into-k-bins-what-is-the-probability-that-every-bin-gets-a

$P(X=0)=\sum_{j=0}^k (-1)^j {k\choose j}\left(1-{j\over k}\right)^n$

In [4]:
def calc(k):
    n = 3* k - 1
    return 1-sum([pow(-1,h)*comb(k,h)*(1-h/k)**n for h in range(k)])

flag = 0
i=1
while flag==0:
    i+=1
    n=i*3    
    if calc(i) > 1/3:
        print("For {} rounds the chance is {:.6f} ".format(i,calc(i)))
        flag=1

For 8 rounds the chance is 0.334578 


In [5]:
url='https://www.janestreet.com/puzzles/robot-swimming-trials-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
HTML(str(soup.find('div', {'class' :'inner-wrapper'})))

In [6]:
# Check with some simulations
@nb.njit()
def sim(N):
    picks = [randrange(1, N+1) for p in range(N*3-1)]
    for n in range(1,N+1):
        if n not in picks:
            return 1
    return 0

@nb.njit()
def run_it(N,sims):
    tot = 0
    for x in range(sims):
        tot += sim(N)
    return tot/sims


In [7]:
start= time.time()
print("Rounds\tSim\t\tCalc")
for N in range(2,10):
    sims = 10000000
    x = run_it(N,sims)
    print("{:>3}\t{:.6f}\t{:.6f}".format(N,x,calc(N)))
print("took",time.time()-start)

Rounds	Sim		Calc
  2	0.062528	0.062500
  3	0.116596	0.116598
  4	0.166131	0.166012
  5	0.212186	0.212093
  6	0.255354	0.255368
  7	0.296034	0.296128
  8	0.334598	0.334578
  9	0.370773	0.370878
took 52.159483194351196
